In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import os

# --- Configuration (Should match your training script) ---
NUM_FREQ_POINTS = 100
DATA_FILE_PATH = f"./RC_lowpass_data.npz"
MODEL_PATH = "best_rc_model_cnn.pth"

# --- Re-define the Model and Dataset classes here for a standalone script ---
# (You could also import them if your files are structured as a module)

class CNN1D(nn.Module):
    """
    The CNN model definition MUST match the one used for training.
    """
    def __init__(self, output_size):
        super(CNN1D, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=7, padding=3),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
        )
        self.flatten = nn.Flatten()
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, NUM_FREQ_POINTS)
            dummy_output = self.conv_layers(dummy_input)
            flattened_size = dummy_output.shape[1] * dummy_output.shape[2]
        self.dense_layers = nn.Sequential(
            nn.Linear(flattened_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, output_size)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.dense_layers(x)
        return x

class FunctionalDataset(Dataset):
    """
    We only need this class to easily get the mean and std from the training data.
    """
    def __init__(self, data_path):
        data = np.load(data_path)
        features_np = 20 * np.log10(data['X'] + 1e-8) if data['X'].max() > 1.0 else data['X']
        features_np = features_np.astype(np.float32)
        
        self.features = torch.from_numpy(features_np).unsqueeze(1)
        self.feature_mean = self.features.mean(dim=[0, 2], keepdim=True)
        self.feature_std = self.features.std(dim=[0, 2], keepdim=True)

    def get_scaling_params(self):
        return self.feature_mean, self.feature_std

# --- Main Test Function ---

def test_single_frequency(omega0_true):
    """Tests the model with a single, known cutoff frequency."""
    
    print(f"--- Testing with True ω₀ = {omega0_true:.2e} rad/s ---")

    # 1. Load the model
    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError(f"Model not found at '{MODEL_PATH}'. Please train the model first.")
    
    model = CNN1D(output_size=1)
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval() # Set model to evaluation mode
    print("Loaded trained CNN model.")

    # 2. Get normalization parameters from the training dataset
    if not os.path.exists(DATA_FILE_PATH):
         raise FileNotFoundError(f"Data file not found at '{DATA_FILE_PATH}'. Please generate it first.")

    # This part is a bit inefficient but ensures we use the exact same scaling
    temp_dataset = FunctionalDataset(DATA_FILE_PATH)
    feature_mean, feature_std = temp_dataset.get_scaling_params()

    # 3. Generate the test frequency response
    freqs = np.logspace(2, 6, num=NUM_FREQ_POINTS)
    omega = 2 * np.pi * freqs
    magnitude = np.abs(1 / (1 + 1j * (omega / omega0_true)))

    # 4. Preprocess the input EXACTLY like the training data
    # a) Convert to dB
    magnitude_db = 20 * np.log10(magnitude + 1e-8)
    
    # b) Convert to tensor
    input_tensor = torch.tensor(magnitude_db, dtype=torch.float32)
    
    # c) Add batch and channel dimensions for the CNN: (100,) -> (1, 1, 100)
    input_tensor = input_tensor.unsqueeze(0).unsqueeze(0)
    
    # d) Normalize using the training set's mean and std
    input_normalized = (input_tensor - feature_mean) / (feature_std + 1e-8)

    # 5. Make the prediction
    with torch.no_grad():
        # The model predicts the LOG of omega0
        predicted_log_omega0 = model(input_normalized)
        
        # We need to take the exponent to get the actual value
        predicted_omega0 = torch.exp(predicted_log_omega0)

    # 6. Print results
    print(f"\nRESULTS:")
    print(f"True cutoff frequency (ω₀):      {omega0_true:.4e} rad/s")
    print(f"Predicted cutoff frequency (ω₀): {predicted_omega0.item():.4e} rad/s")
    
    error_percent = 100 * abs(predicted_omega0.item() - omega0_true) / omega0_true
    print(f"Error: {error_percent:.2f}%")


if __name__ == '__main__':
    # Test with a frequency in the middle of our training range
    test_omega0 = 1e5 
    test_single_frequency(test_omega0)

    print("\n" + "="*50 + "\n")

    # Test with a frequency near the lower end
    test_omega0_low = 5e3
    test_single_frequency(test_omega0_low)

--- Testing with True ω₀ = 1.00e+05 rad/s ---
Loaded trained CNN model.

RESULTS:
True cutoff frequency (ω₀):      1.0000e+05 rad/s
Predicted cutoff frequency (ω₀): 9.8626e+04 rad/s
Error: 1.37%


--- Testing with True ω₀ = 5.00e+03 rad/s ---
Loaded trained CNN model.

RESULTS:
True cutoff frequency (ω₀):      5.0000e+03 rad/s
Predicted cutoff frequency (ω₀): 4.9864e+03 rad/s
Error: 0.27%
